# Explicit matrix factorization based on Keras 2

The idea is based on embeddings, as explained by Jeremy Howard of [fast.ai](http://course.fast.ai/lessons/lesson4.html).

The main idea is to apply embeddings to a problem like user-movie ratings (content filtering), by using a matrix decomposition.  The particular method has to be able to deal with missing values in the matrix, so SVD or PCA cannot work here.

The particular inspiration is from a [blog](http://www.albertauyeung.com/post/python-matrix-factorization/), modified to use Keras v2.

The code for the final application is taken from [3rd place winner](https://github.com/entron/entity-embedding-rossmann) of a similar [competition](https://www.kaggle.com/c/rossmann-store-sales/) on Kaggle.  I plan to use the embeddings format to plug into that type of model later.

This example code below deal with an example problem using a tiny matrix, just to get a handle on the options.

In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
%config Completer.use_jedi = False


In [2]:
from keras.models import Model as KerasModel
from keras.layers import Input, Reshape, Dot
from keras.layers.embeddings import Embedding
from keras import optimizers


Using TensorFlow backend.


I use a sample matrix, with `0` representing a missing value, exactly the same as in inspiration blog.

In [3]:
ds = pd.DataFrame([
    [5, 3, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [1, 0, 0, 0],
    [0, 1, 5, 4],
])
ds.replace(to_replace=0,value=np.nan,inplace=True)
ds

,0,1,2,3
0,5.0,3.0,NaN,1.0
1,4.0,NaN,NaN,1.0
2,1.0,1.0,NaN,5.0
3,1.0,NaN,NaN,NaN
4,NaN,1.0,5.0,4.0


Per Andrew Ng's [suggestion](https://www.coursera.org/learn/machine-learning/lecture/Adk8G/about), I could use the "mean normalization" step, but will not do for now for clarity.

In [4]:
col_means=ds.mean(axis=1)
ds_demeaned=ds.subtract(col_means,axis=0)


In [5]:
ds_unstacked=ds.unstack().reset_index()
ds_unstacked.rename(axis=1,mapper={'level_0':'column','level_1':'row',0:'truth'},inplace=True)

train=ds_unstacked.dropna()
train

,column,row,truth
0,0,0,5.0
1,0,1,4.0
2,0,2,1.0
3,0,3,1.0
5,1,0,3.0
7,1,2,1.0
9,1,4,1.0
14,2,4,5.0
15,3,0,1.0
16,3,1,1.0


I will train 2-dimensional embeddings, meaning each level of `row` and `column` will be represented by 2 floating point numbers.  This is lower dimensionality than the `row` and `column` cardinality (4, and 5)

In [6]:
n_embed = 2
n_levels=train.nunique()
n_levels

column    4
row       5
truth     4
dtype: int64

The format of the Keras model matches that of the Rossmann competition winner's, so I that I can use the weights later in a larger model.

Note: I want to add regularization to the `Embedding()` layers later.

In [7]:
input_0 = Input(shape=(1,))
output_0 = Embedding(n_levels.column, n_embed, name='embedding_0')(input_0)
output_0 = Reshape(target_shape=(n_embed,))(output_0)

input_1 = Input(shape=(1,))
output_1 = Embedding(n_levels.row, n_embed, name='embedding_1')(input_1)
output_1 = Reshape(target_shape=(n_embed,))(output_1)

This is the core of the model, the single `Dot()` layer from Keras.

In [8]:
output_model = Dot(axes=(1,1), normalize=False)([output_0,output_1])

model = KerasModel(inputs=[input_0,input_1], outputs=output_model)

This model expect a list of arrays as input, not sure if this format is very efficient, but don't know how to improve right now.

In [9]:
x=list(train.iloc[:,:2].values.T)
y=train.iloc[:,2]
x

[array([0, 0, 0, 0, 1, 1, 1, 2, 3, 3, 3, 3]),
 array([0, 1, 2, 3, 0, 2, 4, 4, 0, 1, 2, 4])]

Had to restart `model.fit()` several times to get a good result, usually final loss can go down to ~`0.01` with 2 dimensions.

In [10]:
sgd = optimizers.SGD(lr=0.4, decay=1e-6, momentum=0.5, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd)
model.fit(x,y,epochs=10)

Epoch 1/10
12/12 [==============================] - 0s 6ms/step - loss: 10.1678
Epoch 2/10
12/12 [==============================] - 0s 186us/step - loss: 10.1620
Epoch 3/10
12/12 [==============================] - 0s 159us/step - loss: 10.1503
Epoch 4/10
12/12 [==============================] - 0s 256us/step - loss: 10.1109
Epoch 5/10
12/12 [==============================] - 0s 357us/step - loss: 9.9738
Epoch 6/10
12/12 [==============================] - 0s 174us/step - loss: 9.4997
Epoch 7/10
12/12 [==============================] - 0s 353us/step - loss: 7.9705
Epoch 8/10
12/12 [==============================] - 0s 226us/step - loss: 4.2840
Epoch 9/10
12/12 [==============================] - 0s 200us/step - loss: 1.4216
Epoch 10/10
12/12 [==============================] - 0s 227us/step - loss: 0.5594


I can look at the weights, they don't seem to get excessively large without regularization right now.

In [11]:
model.get_layer('embedding_0').get_weights()[0]

array([[-0.31470114, -2.45210218],
       [-0.23043904, -1.12569129],
       [-2.0864985 , -0.7506308 ],
       [-2.02308679, -0.29103282]], dtype=float32)

In [12]:
model.get_layer('embedding_1').get_weights()[0]

array([[-0.23636681, -2.09086394],
       [-0.2060218 , -1.61287391],
       [-1.94337177, -0.3037861 ],
       [-0.10028759, -0.41946062],
       [-2.14217663, -0.69291216]], dtype=float32)

I will predict and compare to the true values.  Note the predicted matrix does not have any missing, allowing us to achieve the goal of the competition.

In [13]:
ds_unstacked['predicted']=model.predict(list(ds_unstacked.iloc[:,:2].values.T))
ds_unstacked.predicted=ds_unstacked.predicted.apply(lambda x: round(x,2))

ds_unstacked.pivot_table(values=['predicted','truth'],index='row',columns='column')

predicted                   truth               
column         0     1     2     3     0    1    2    3
row                                                    
0           5.20  2.41  2.06  1.09   5.0  3.0  NaN  1.0
1           4.02  1.86  1.64  0.89   4.0  NaN  NaN  1.0
2           1.36  0.79  4.28  4.02   1.0  1.0  NaN  5.0
3           1.06  0.50  0.52  0.32   1.0  NaN  NaN  NaN
4           2.37  1.27  4.99  4.54   NaN  1.0  5.0  4.0

## Conclusion
I can train embeddings for item-item combinations, even with some combinations missing.  The embedding layers can be used later in larger model, or weights can be continuous variables in other models.